### 4.8 Task 

#### 1. Create a new notebook for this task. Be sure to import the relevant libraries, along with your ords_prods_merge dataframe, which should include your newly derived columns from the previous Exercise.

In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
project_path = r'C:\Users\cneva\06-2024 Instacart Basket Analysis'
df_ords_prods = pd.read_pickle(os.path.join(project_path, '02 Data', 'Prepared Data', 'ords_prods_3.pkl'))

In [5]:
df_ords_prods.shape

(32404859, 18)

In [6]:
df_ords_prods.head(1)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_day_2,busiest_period_of_day
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy days,Most orders


#### 2. In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [8]:
df_ords_prods.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


#### 3. Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.

##### The subset didnt include all departments. Because the dataframe contains much more data, some departments have gone up and some have gone down

#### 4. Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

In [12]:
# crate max order per user_id column
df_ords_prods['max_order'] = df_ords_prods.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\cneva\AppData\Local\Temp\ipykernel_12616\2247696144.py:2: FutureWarning: The provided callable <function max at 0x0000029B37C36AC0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_ords_prods['max_order'] = df_ords_prods.groupby(['user_id'])['order_number'].transform(np.max)


In [13]:
df_ords_prods.head(2)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_day_2,busiest_period_of_day,max_order
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy days,Most orders,10
1,2539329,1,1,2,8,NaN,True,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,Mid-range product,Regularly busy,Regularly busy days,Most orders,10


In [15]:
# create loyalty category column: 
# loyal customer = max order > 40
# regular customer = 10 < max order <= 40
# new customer = max order <= 10
df_ords_prods.loc[df_ords_prods['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [17]:
df_ords_prods.loc[(df_ords_prods['max_order'] > 10) & (df_ords_prods['max_order'] <= 40), 'loyalty_flag'] = 'Regular customer'

In [19]:
df_ords_prods.loc[df_ords_prods['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [23]:
df_ords_prods.head(5)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_day_2,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy days,Most orders,10,New customer
1,2539329,1,1,2,8,NaN,True,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,Mid-range product,Regularly busy,Regularly busy days,Most orders,10,New customer
2,2539329,1,1,2,8,NaN,True,12427,3,0,Original Beef Jerky,23,19,4.4,Low-range product,Regularly busy,Regularly busy days,Most orders,10,New customer
3,2539329,1,1,2,8,NaN,True,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,Low-range product,Regularly busy,Regularly busy days,Most orders,10,New customer
4,2539329,1,1,2,8,NaN,True,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,Low-range product,Regularly busy,Regularly busy days,Most orders,10,New customer


In [25]:
df_ords_prods['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

#### 5. The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [27]:
# groupby loyalty flag and aggregate mean, min and max
df_ords_prods.groupby('loyalty_flag').agg({'prices':['mean','min','max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

##### On average, new customers purchase higher priced products while loyal customer purchase the lower priced products

#### 6. The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:
##### - If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
##### - If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [42]:
# Groupby user_id and aggregate by average price
df_ords_prods['user_average_price'] = df_ords_prods.groupby('user_id')['prices'].transform(np.mean)

C:\Users\cneva\AppData\Local\Temp\ipykernel_12616\468347269.py:2: FutureWarning: The provided callable <function mean at 0x0000029B37C374C0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_ords_prods['user_average_price'] = df_ords_prods.groupby('user_id')['prices'].transform(np.mean)


In [46]:
# Check 2 users
df_ords_prods[df_ords_prods['user_id'].isin([99,9999])]

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,...,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_day_2,busiest_period_of_day,max_order,loyalty_flag,user_average_price
12714,2736359,99,1,4,15,NaN,True,27845,1,0,...,84,16,12.8,Mid-range product,Least busy,Least busy days,Most orders,18,Regular customer,7.958294
12715,2736359,99,1,4,15,NaN,True,23516,2,0,...,17,13,9.7,Mid-range product,Least busy,Least busy days,Most orders,18,Regular customer,7.958294
12716,2736359,99,1,4,15,NaN,True,23726,3,0,...,17,13,1.7,Low-range product,Least busy,Least busy days,Most orders,18,Regular customer,7.958294
12717,2736359,99,1,4,15,NaN,True,23405,4,0,...,17,13,1.3,Low-range product,Least busy,Least busy days,Most orders,18,Regular customer,7.958294
12718,2736359,99,1,4,15,NaN,True,43920,5,0,...,17,13,12.0,Mid-range product,Least busy,Least busy days,Most orders,18,Regular customer,7.958294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1542538,1516105,9999,6,0,14,17.0,False,13424,2,0,...,45,19,4.2,Low-range product,Busiest day,Busiest days,Most orders,9,New customer,7.728571
1542539,1516105,9999,6,0,14,17.0,False,6184,3,0,...,32,4,4.3,Low-range product,Busiest day,Busiest days,Most orders,9,New customer,7.728571
1542540,2555710,9999,7,0,7,7.0,False,31759,1,0,...,121,14,2.7,Low-range product,Busiest day,Busiest days,Fewest orders,9,New customer,7.728571
1542541,1978102,9999,8,0,13,30.0,False,46799,1,0,...,72,13,7.2,Mid-range product,Busiest day,Busiest days,Most orders,9,New customer,7.728571


In [48]:
# flag user based on user_average_price
# lower spender = user_average_price < 10
# high spender = user_avereage_price >= 10
df_ords_prods.loc[df_ords_prods['user_average_price'] < 10, 'user_spender_type'] = 'Low spender'

In [52]:
df_ords_prods.loc[df_ords_prods['user_average_price'] >= 10, 'user_spender_type'] = 'High spender'

In [54]:
# check flag
df_ords_prods.head(3)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,...,department_id,prices,price_range_loc,busiest_day,busiest_day_2,busiest_period_of_day,max_order,loyalty_flag,user_average_price,user_spender_type
0,2539329,1,1,2,8,NaN,True,196,1,0,...,7,9.0,Mid-range product,Regularly busy,Regularly busy days,Most orders,10,New customer,6.367797,Low spender
1,2539329,1,1,2,8,NaN,True,14084,2,0,...,16,12.5,Mid-range product,Regularly busy,Regularly busy days,Most orders,10,New customer,6.367797,Low spender
2,2539329,1,1,2,8,NaN,True,12427,3,0,...,19,4.4,Low-range product,Regularly busy,Regularly busy days,Most orders,10,New customer,6.367797,Low spender


In [58]:
df_ords_prods['user_spender_type'].value_counts(dropna = False)

user_spender_type
Low spender     31770614
High spender      634245
Name: count, dtype: int64

#### 7. In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:
##### - If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
##### - If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
##### - If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [63]:
# groupby user_id and aggregate days_since_prior_order by the median
df_ords_prods['user_median_days_since'] = df_ords_prods.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\cneva\AppData\Local\Temp\ipykernel_12616\728385213.py:2: FutureWarning: The provided callable <function median at 0x0000029B37D63380> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  df_ords_prods['user_median_days_since'] = df_ords_prods.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [65]:
# flag user based on user_average_price
# Non-frequent customer = user_median_days_since > 20
# Regular customer = 10 < user_median_days_since <= 20
#  Frequent customer = user_median_days_since <= 10
df_ords_prods.loc[df_ords_prods['user_median_days_since'] > 20, 'user_order_frequency_flag'] = 'Non-frequent customer'

In [67]:
df_ords_prods.loc[(df_ords_prods['user_median_days_since'] <= 20) & (df_ords_prods['user_median_days_since'] > 10), 'user_order_frequency_flag'] = 'Regular customer'

In [69]:
df_ords_prods.loc[df_ords_prods['user_median_days_since'] <= 10, 'user_order_frequency_flag'] = 'Frequent customer'

In [71]:
df_ords_prods.head(3)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest_day,busiest_day_2,busiest_period_of_day,max_order,loyalty_flag,user_average_price,user_spender_type,user_median_days_since,user_order_frequency_flag
0,2539329,1,1,2,8,NaN,True,196,1,0,...,Mid-range product,Regularly busy,Regularly busy days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2539329,1,1,2,8,NaN,True,14084,2,0,...,Mid-range product,Regularly busy,Regularly busy days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,2539329,1,1,2,8,NaN,True,12427,3,0,...,Low-range product,Regularly busy,Regularly busy days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [75]:
df_ords_prods['user_order_frequency_flag'].value_counts()

user_order_frequency_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
Name: count, dtype: int64

#### 8. Ensure your notebook is clean and structured and that your code is well commented.

#### 9. Export your dataframe as a pickle file and store it correctly in your “Prepared Data” folder.

In [78]:
df_ords_prods.to_pickle(os.path.join(project_path,'02 Data','Prepared Data','ords_prods_4.pkl'))

#### 10. Save your notebook and submit it to your tutor for review.